In [27]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from datetime import datetime, date
import fm_data_scraper as scraper
import json

In [16]:
scraper.get_abilities('https://fmdataba.com/19/p/578/keylor-navas/', True)

{'goalkeeping': {'Aerial Reach': 10,
  'Command Of Area': 15,
  'Communication': 14,
  'Eccentricity': 4,
  'First Touch': 8,
  'Handling': 12,
  'Kicking': 13,
  'One On Ones': 16,
  'Passing': 9,
  'Reflexes': 19,
  'Rushing Out': 12,
  'Tendency To Punch': 16,
  'Throwing': 15},
 'mental': {'Aggression': 11,
  'Anticipation': 15,
  'Bravery': 15,
  'Composure': 13,
  'Concentration': 18,
  'Decisions': 15,
  'Determination': 16,
  'Flair': 8,
  'Leadership': 12,
  'Off The Ball': 1,
  'Positioning': 14,
  'Teamwork': 15,
  'Vision': 8,
  'Work Rate': 15},
 'physical': {'Acceleration': 12,
  'Agility': 18,
  'Balance': 14,
  'Jumping Reach': 12,
  'Natural Fitness': 17,
  'Pace': 14,
  'Stamina': 14,
  'Strength': 13}}

In [17]:
# parse player urls on a single league

# url = 'https://fmdataba.com/19/l/4/premier-league/best-players/'

urls = ['https://fmdataba.com/19/l/4/premier-league/best-players/']

with open('fm_urls.txt', 'w') as f:
    for url in urls:
        for i in range(12):
            url2 = url + str(i)
            r = requests.get(url2, headers = {'User-Agent':'Mozilla/5.0'})
            soup = bs(r.content, 'html.parser')
            rows = soup.find("table").find("tbody").find_all("tr")
            for r in rows:
                if r.find_all('a', href=True):
                    f.write(r.find_all('a', href=True)[1]['href'] + '\n')

In [18]:
test_data = scraper.get_profile_data('https://fmdataba.com/19/p/4592/son-heung-min/')

In [19]:
# test_data['abilities'] = scraper.get_abilities('https://fmdataba.com/19/p/4592/son-heung-min/')

In [20]:
test_data

{'profile_img': 'https://fmdataba.com/images/p/4592.png',
 'name': 'Son Heung-Min ',
 'birth_date': '8/7/1992',
 'nation': 'South Korea KOR',
 'position': 'M (R), AM (RL), ST (C)',
 'foot': 'Either'}

In [21]:
test_data2 = scraper.get_profile_data('https://fmdataba.com/19/p/1103/marc-andre-ter-stegen/')

In [22]:
test_data2

{'profile_img': 'https://fmdataba.com/images/p/1103.png',
 'name': 'Marc-André ter Stegen ',
 'birth_date': '30/4/1992',
 'nation': 'Germany',
 'position': 'GK',
 'foot': 'Either'}

In [23]:
with open('data.json', 'w') as json_file:  
    json.dump([test_data, test_data2], json_file, indent=2)

In [41]:
def get_profile_data(url):
    """Gets player's profile information based on the URL link given

    Parameters
    ----------
    url : str
        The URL of the player data e.g. 'https://fmdataba.com/19/p/1165/lionel-messi/'

    Returns
    -------
    dict
        a dict that contains the player profile information (name, nationality, birth date, position, and preferred foot)
    """

    profile_data = {}
    page = requests.get(url, headers={'User-Agent':'Mozilla/5.0'})
    soup = bs(page.content, 'html.parser')
    rows = soup.find("table").find("tbody").find_all("tr")[0]
    #print(pd.read_html(page.text))
    tables = pd.read_html(page.text)[0]
    #print(tables)
    #idx = tables[1][12].index('Main')
    img_meta = rows.findAll('img')[0]
    idx2 = img_meta['alt'].index('FM')
    profile_data['profile_img'] = img_meta['src']
    profile_data['name'] = img_meta['alt'][:idx2]
    #print(profile_data['name'])
    profile_data['birth_date'] = tables[1][8].split(' ')[0].strip('()')
    profile_data['nation'] = tables[1][12]#[:idx]
    profile_data['position'] = tables[1][21]
    profile_data['foot'] = tables[1][19]

    return profile_data

In [42]:
with open('fm_urls.txt') as f:
    data_list = []
    for url in f:
        data = get_profile_data(url)
        data['abilities'] = scraper.get_abilities(url)
        data_list.append(data)
        
with open('data.json', 'w') as json_file:  
    json.dump(data_list, json_file, indent=2)

KeyError: 21